<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Construction des caractéristiques ou *features*

Les données textuelles ne peuvent pas être utilisés directment dans les différents algorithmes de d'apprentissage statistiques. Nous allons voir dans ce tutoriel plusieurs technique permettant de traduires les données textuelles sous formes de vecteur numérique : 


Nombresue fonction de vectorisation présente dans scikit-learn :

* `One-Hot-Encoder`
* `Tf-Idf`
* `Hashing`

Word Embedding dans la librairies gensim :

* `Word2Vec`


## Téléchargement des librairies

In [1]:
#Importation des librairies utilisées
import time
import pandas as pd
import numpy as np
import collections
import itertools
import os
import warnings
warnings.filterwarnings('ignore')


from sklearn.cross_validation import train_test_split

/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Téléchargement des données

In [2]:
data_valid_clean_stem = pd.read_csv("data/cdiscount_valid_clean_stem.csv").fillna("")
data_train_clean_stem = pd.read_csv("data/cdiscount_train_clean_stem.csv").fillna("")

On créé un dossier dans lequel nous allons sauvegarder les DataFrame constitués des features que l'on va construire dans ce notebook 

In [3]:
DATA_OUTPUT_DIR = "data/features"
if not(os.path.isdir("data/features")):
    os.mkdir("data/features")

Dans un premier temps on considère seulement la colonne *Description* de nos `DataFrame`

In [4]:
train_array = data_train_clean_stem["Description"].values
valid_array = data_valid_clean_stem["Description"].values

## Vectorisaton

### One-Hot-Encoding

* **One-Hot-Encoding** ...

possibilité de ngram/Decrire les paramèttr

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

extr_cv = CountVectorizer(binary=False)
data_train_OHE = extr_cv.fit_transform(train_array)
data_train_OHE

<95000x56578 sparse matrix of type '<class 'numpy.int64'>'
	with 1118684 stored elements in Compressed Sparse Row format>

In [6]:
vocabulary = extr_cv.get_feature_names()
N_vocabulary = len(vocabulary)
N_vocabulary

56578

Affiche la première ligne

In [7]:
ir = 0

rw = data_train_OHE.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vocabulary[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","weight"])


spac play clark alison spac play clark alison voir present


,indices,token,weight
0,41158,present,1
1,54390,voir,1
2,1463,alison,2
3,10365,clark,2
4,39980,play,2
5,48027,spac,2


In [8]:
data_valid_OHE = extr_cv.transform(valid_array)
data_valid_OHE

<5000x56578 sparse matrix of type '<class 'numpy.int64'>'
	with 57336 stored elements in Compressed Sparse Row format>

In [9]:
ir = 5

rw = data_valid_OHE.getrow(ir)
print(valid_array[ir])
pd.DataFrame([(v, vocabulary[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","weight"])


tuyau arrosag flexiblecaracterist doubl couch sup fort durabl ultra leg peut absolu tordr etir fois


,indices,token,weight
0,272,absolu,1
1,3109,arrosag,1
2,12212,couch,1
3,15850,doubl,1
4,16364,durabl,1
5,18743,etir,1
6,20661,fois,1
7,20859,fort,1
8,29823,leg,1
9,39292,peut,1


### TF-IDF¶

* **TF-IDF**. Le TF-IDF permet de faire ressortir l'importance relative de chaque mot $m$ (ou couples de mots consécutifs) dans un texte-produit ou un descriptif $d$, par rapport à la liste entière des produits. La fonction $TF(m,d)$ compte le nombre d'occurences du mot $m$ dans le descriptif $d$. La fonction $IDF(m)$ mesure l'importance du terme dans l'ensemble des documents ou descriptifs en donnant plus de poids aux termes les moins fréquents car considérés comme les plus discriminants (motivation analogue à celle de la métrique du chi2 en anamlyse des correspondance). $IDF(m,l)=\log\frac{D}{f(m)}$ où $D$ est le nombre de documents, la taille de l'échantillon d'apprentissage, et $f(m)$ le nombre de documents ou descriptifs contenant le mot $m$. La nouvelle variable ou *features* est $V_m(l)=TF(m,l)\times IDF(m,l)$.

* Comme pour les transformations des variables quantitatives (centrage, réduction), la même transformation c'est-à-dire les mêmes pondérations, est calculée sur l'achantillon d'apprentissage et appliquée à celui de test.

On utiliser la fonction TfidfVectorizer qui permet de parser également le texte

On fixe le paramètre `norm` = False pour rendre les résultats plus explicite 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer( ngram_range=(1,1), norm = False)
data_train_TFIDF = vec.fit_transform(train_array)
data_train_TFIDF

<95000x56578 sparse matrix of type '<class 'numpy.float64'>'
	with 1118684 stored elements in Compressed Sparse Row format>

In [11]:
vocabulary = vec.get_feature_names()
N_vocabulary = len(vocabulary)
N_vocabulary

56578

In [12]:
ir = 0

rw = data_train_TFIDF.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vocabulary[v], vec.idf_[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","idf","weight"])


spac play clark alison spac play clark alison voir present


,indices,token,idf,weight
0,48027,spac,9.060445,18.120891
1,39980,play,8.302760,16.605519
2,10365,clark,9.628429,19.256859
3,1463,alison,9.976736,19.953472
4,54390,voir,1.334998,1.334998
5,41158,present,1.341449,1.341449


Commentez, Comment evolue les poids en changeant les paramètre smooth idf, sublinear_tf?

En cchangean le nombre de ngram?

On applique maintenant le `vectorizer` sur le jeu de données de validation

In [13]:
data_valid_TFIDF = vec.transform(valid_array)
data_valid_TFIDF

<5000x56578 sparse matrix of type '<class 'numpy.float64'>'
	with 57336 stored elements in Compressed Sparse Row format>

Si le tf est recalculé pour chaque ligne, le même idf est utilisé

In [14]:
ir = 5

rw = data_valid_TFIDF.getrow(ir)
print(valid_array[ir])
pd.DataFrame([(v, vocabulary[v], vec.idf_[v], k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","token","idf","weight"])


tuyau arrosag flexiblecaracterist doubl couch sup fort durabl ultra leg peut absolu tordr etir fois


,indices,token,idf,weight
0,52702,ultra,3.430309,3.430309
1,52461,tuyau,8.198963,8.198963
2,49279,sup,6.699591,6.699591
3,39292,peut,6.347961,6.347961
4,29823,leg,6.354620,6.354620
5,20859,fort,7.418218,7.418218
6,20661,fois,6.518843,6.518843
7,18743,etir,9.976736,9.976736
8,16364,durabl,7.143523,7.143523
9,15850,doubl,5.823075,5.823075


### Hashing

* **Hashage**. Il permet de réduire l'espace des variables (taille du dictionnaire) en un nombre limité et fixé a priori `n_hash` de caractéristiques. Il repose sur la définition d'une fonction de hashage, $h$ qui à un indice $j$ défini dans l'espace des entiers naturels, renvoie un indice $i=h(j)$ dans dans l'espace réduit (1 à n_hash) des caractéristiques. Ainsi le poids de l'indice $i$, du nouvel espace, est l'association de tous les poids d'indice $j$ tels que $i=h(j)$ de l'espace originale. Ici, les poids sont associés d'après la méthode décrite par Weinberger et al. (2009).

N.B. $h$ n'est pas généré aléatoirement. Ainsi pour un même fichier d'apprentissage (ou de test) et pour un même entier n_hash, le résultat de la fonction de hashage est identique

Take a Dictionnary as an input

In [15]:
from sklearn.feature_extraction import FeatureHasher
nb_hash = 300

feathash = FeatureHasher(nb_hash)
train_dict_array  = map(lambda x : collections.Counter(x.split(" ")), train_array)
data_train_hash = feathash.fit_transform(train_dict_array)

In [16]:
data_train_hash


<95000x300 sparse matrix of type '<class 'numpy.float64'>'
	with 1092893 stored elements in Compressed Sparse Row format>

In [17]:

ir = 0

rw = data_train_hash.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, k)  for k,v in zip(rw.data,rw.indices)], columns=["indices","weight"])


spac play clark alison spac play clark alison voir present


,indices,weight
0,18,2.0
1,82,-2.0
2,200,-2.0
3,216,-2.0
4,220,-1.0
5,274,1.0


Reduit la taille de la matrice, MAIS pas de fonction inverse transform. 
Resultat beaucoup moin facilement compréhensible
Efficace que si vraient beaucoup de ligne (>100.000)

Il est ensuite possible de passer la nouvelle matrice *hasher* (`data_train_hash`) directement dans un algorithme d'apprentissage, ou bien de combiner ces poids avec une fonction de TFIDF

Cette fois la fonction `TFIDFTransformer`est utilisé, ne considère pas les string, mais les tf comme les poids présent

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer

vec =  TfidfTransformer(norm = False)
data_train_HTfidf = vec.fit_transform(data_train_hash)
data_train_HTfidf

<95000x300 sparse matrix of type '<class 'numpy.float64'>'
	with 1084522 stored elements in Compressed Sparse Row format>

In [19]:
ir = 0

rw = data_train_HTfidf.getrow(ir)
print(train_array[ir])
pd.DataFrame([(v, vec.idf_[v], k)  for k,v in zip(rw.data, rw.indices)], columns=["indices","idf_","weight"])


spac play clark alison spac play clark alison voir present


,indices,idf_,weight
0,274,1.310749,1.310749
1,220,1.326776,-1.326776
2,216,4.443017,-8.886034
3,200,4.879923,-9.759846
4,82,4.685108,-9.370215
5,18,4.540833,9.081666


### Save Vectorize Vector

De nombreux paramètres a regler entraine donc un un très grand nombre de combinaison.

ici nous créer seulement par défault 4 jeu de données avec count et TFIDF chacun avec et sans hashin de taille 300.

A vous de jouer sur d'autre paramètre nombre de gram, taille de la fonction de hashing, colonne pris en compte ... pour générer d'autre dataset et tester leur influence

### Fonction de Vectorisation

on créé deux fonctions `vectorizer_train` and `apply_vectorizer` afin de générer automatiquement différent dataframe d'apprentissage et de validation vectorisé.

In [20]:
def vectorizer_train(df, columns=['Description'], nb_hash=None, nb_gram = 1, vectorizer = "tfidf" , binary = False):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    # Hashage
    if nb_hash is None:
        feathash = None
        if vectorizer == "tfidf":
            vec = TfidfVectorizer(ngram_range=(1,nb_gram))
            data_vec = vec.fit_transform(data_array)
        else:
            vec = CountVectorizer(binary=binary)
            data_vec = vec.fit_transform(data_array)
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        feathash = FeatureHasher(nb_hash)
        data_hash = feathash.fit_transform(data_dic_array)
        
        if vectorizer=="tfidf":
            vec =  TfidfTransformer()
            data_vec =  vec.fit_transform(data_hash)
        else:
            vec = None
            data_vec = data_hash

    return vec, feathash, data_vec



def apply_vectorizer(df, vec, feathash, columns =['Description', 'Libelle', 'Marque']):
    
    data_array = [" ".join(line) for line in df[columns].values]
    
    #Hashage
    if feathash is None:
        data_hash = data_array
    else:
        data_dic_array = [collections.Counter(line.split(" ")) for line in data_array]
        data_hash = feathash.transform(data_dic_array)
    
    if vec is None:
        data_vec = data_hash
    else:
        data_vec = vec.transform(data_hash)
    return data_vec


In [21]:
parameters = [[None, "count"],
              [300, "count"],
              [10000,"count"],
              [None, "tfidf"],
              [300, "tfidf"],
             [10000,"tfidf"]]

from scipy import sparse

for nb_hash, vectorizer in parameters:
    ts = time.time()
    vec, feathash, data_train_vec = vectorizer_train(data_train_clean_stem, nb_hash=nb_hash, vectorizer = vectorizer)
    data_valid_vec = apply_vectorizer(data_valid_clean_stem, vec, feathash)
    te = time.time()
    
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    print("Runing time for vectorization : %.1f seconds" %(te-ts))
    print("Train shape : " + str(data_train_vec.shape))
    print("Valid shape : " + str(data_valid_vec.shape))

    
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_train_vec)
    sparse.save_npz(DATA_OUTPUT_DIR +"/vec_valid_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer), data_valid_vec)
    

nb_hash : None, vectorizer : count
Runing time for vectorization : 2.2 seconds
Train shape : (95000, 56578)
Valid shape : (5000, 56578)
nb_hash : 300, vectorizer : count
Runing time for vectorization : 1.8 seconds
Train shape : (95000, 300)
Valid shape : (5000, 300)
nb_hash : 10000, vectorizer : count
Runing time for vectorization : 1.9 seconds
Train shape : (95000, 10000)
Valid shape : (5000, 10000)
nb_hash : None, vectorizer : tfidf
Runing time for vectorization : 2.3 seconds
Train shape : (95000, 56578)
Valid shape : (5000, 56578)
nb_hash : 300, vectorizer : tfidf
Runing time for vectorization : 1.9 seconds
Train shape : (95000, 300)
Valid shape : (5000, 300)
nb_hash : 10000, vectorizer : tfidf
Runing time for vectorization : 1.9 seconds
Train shape : (95000, 10000)
Valid shape : (5000, 10000)


## Word2Vec

In [22]:
import gensim

import nltk 
stemmer=nltk.stem.SnowballStemmer('french')

### Build Word2Vec model

In [23]:
train_array_token = [line.split(" ") for line in train_array]
valid_array_token = [line.split(" ") for line in valid_array]

In [24]:
Features_dimension = 300
sg = 1
hs = 0
negative = 10
X = train_array_token
N_train = len(X)
#sg (int {1, 0}) – Defines the training algorithm. If 1, skip-gram is employed; otherwise, CBOW is used.


#hs (int {1,0}) – If 1, hierarchical softmax will be used for model training. 
#If set to 0, and negative is non-zero, negative sampling will be used.

#negative (int) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). 
#If set to 0, no negative sampling is used

In [25]:
model_dic = {}
for sg in [0,1]:
    print("Start learning Word2Vec learning")
    print("Params are : Fdim_%d_sg_%d_hs_%d_negative_%d_model" %(Features_dimension, sg, hs, negative))
    ts = time.time()
    model = gensim.models.Word2Vec(X, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
    te = time.time()
    t_learning = te-ts

    # Metadata
    N_vocab, feature_dim = model.wv.vectors.shape
    metadata = {"learning_time" : t_learning, "vocab_size" : N_vocab, "sg" : sg, "negative": negative, "hs":hs}
    print(metadata)
    
    model_name = "skip-gram" if sg==1 else "CBOW"
    model_dic.update({model_name : model})

model_online_dir = "data/fr/fr.bin"
model = gensim.models.Word2Vec.load(model_online_dir)
model_dic.update({"online" : model})

Start learning Word2Vec learning
Params are : Fdim_300_sg_0_hs_0_negative_10_model
{'vocab_size': 56579, 'negative': 10, 'hs': 0, 'sg': 0, 'learning_time': 11.873658180236816}
Start learning Word2Vec learning
Params are : Fdim_300_sg_1_hs_0_negative_10_model
{'vocab_size': 56579, 'negative': 10, 'hs': 0, 'sg': 1, 'learning_time': 39.20227599143982}


### Most similar world
Résultats pertinent seulement si N_train=1.000.000, 100.000 trop peu.

In [26]:
model.predict_output_word(["homm"])

In [27]:
term="homme"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : homme


,CBOW,online,skip-gram
0,femm,aventurier,unisex
1,unisex,aristocrate,arman
2,arman,garçon,emporio
3,xinj,vieillard,chronograph
4,diesel,délinquant,hommecollect
5,dachstein,écrivain,army
6,adh,politicien,adh
7,norway,escroc,sportsman
8,geographical,orateur,adid
9,redskin,érudit,pantalon


In [28]:
term="femme"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : femme


,CBOW,online,skip-gram
0,homm,servante,unisex
1,unisex,mère,femmecouleur
2,fashion,fille,femmecollect
3,sandal,belle-mère,dkny
4,ballerin,fiancée,lublin
5,bottin,compagne,arman
6,compense,sœur,leandr
7,contrejour,prostituée,dress
8,escarpin,nourrice,bred
9,bicolor,demi-sœur,marroncomposit


In [29]:
term="xbox"

df_ = []
columns = []
for model_name, model in model_dic.items():
    token = stemmer.stem(term) if "online"!=model_name else term
    mpow = model.wv.most_similar([token])
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
print("Most similar words for word : "+term)
pd.DataFrame(np.array(df_).T, columns=columns)

Most similar words for word : xbox


,CBOW,online,skip-gram
0,subliminal,playstation,playstat
1,fchcmax,gamecube,wii
2,travel,dreamcast,controll
3,scuf,psp,scuf
4,visor,wii,microsoft
5,krakau,neo-geo,unleashed
6,kingdom,mega,wrath
7,nomad,advance,nois
8,unleashed,famicom,cruz
9,embedded,nintendo,scrib


In [30]:
terms_positif = ["femme","roi"]
terms_negatif = ["homme"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,online,skip-gram
0,jungl,reine,bstick
1,lapin,belle-mère,redecouvr
2,savan,brunehilde,tendress
3,calin,régente,jungl
4,lion,reine-mère,effig
5,eleph,belle-sœur,licorn
6,coquelicot,veuve,marionnet
7,romant,mère,ruin
8,canard,demi-sœur,collag
9,amour,nièce,paon


In [31]:
terms_positif = ["espagne","paris"]
terms_negatif = ["france"]

df_ = []
columns = []
for model_name, model in model_dic.items():
    token_positif = [stemmer.stem(term) if "online"!=model_name else term for term in terms_positif]
    token_negativ = [stemmer.stem(term) if "online"!=model_name else term for term in terms_negatif]
    
    mpow = model.wv.most_similar(positive=token_positif, negative=token_negativ)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,online,skip-gram
0,flex,ségovie,orlando
1,inde,madrid,edinburgh
2,canc,alicante,manil
3,normand,aix-en-provence,janeiro
4,gourmandis,ancône,darkf
5,etire,séville,agra
6,liberty,malaga,copenhagen
7,arkans,andalousie,seattl
8,aquarius,alger,cancun
9,cancun,avignon,autrich


### Predict output word

In [32]:
terms = ["voir","la"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,online,skip-gram
0,neuv,ci-dessous,referent
1,present,ci-dessus,appartient
2,mettr,ci-après,sac
3,votr,l'article,lign
4,sou,infra,main
5,presentat,paragraphe,if
6,ave,bibliographie,prix
7,met,liste,reduit
8,infiniteshopping,ci-contre,couleur
9,tou,section,bandoulier


In [33]:
terms = ["coque",'pour',"samsung"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,online,skip-gram
0,galaxy,acier,ace
1,ace,aluminium,galaxy
2,stuff,coque,not
3,not,former,express
4,xcov,béton,cor
5,cor,métallique,soupl
6,trend,protéger,mad
7,sii,remplacer,stuff
8,plai,fabriquer,franc
9,express,conçue,trend


In [34]:
terms = ["homme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,online,skip-gram
0,montr,d'affaires,montr
1,ditl,vieil,chronograph
2,genr,d’affaires,accutron
3,chronograph,politique,genr
4,invict,d'état,norway
5,chemis,d'église,ditl
6,pantalon,cet,chemis
7,vest,jeune,achet
8,vet,néandertal,leman
9,box,d’état,tissot


In [35]:
terms = ["femme"]


df_ = []
columns = []
for model_name, model in model_dic.items():
    tokens = [stemmer.stem(term) if "online"!=model_name else term for term in terms]
    
    mpow = model.predict_output_word(tokens)
    if mpow is None:
        df_.append(["" for k in range(10)])
    else:
        df_.append([k[0] for k in mpow])
    columns.append(model_name)
pd.DataFrame(np.array(df_).T, columns=columns)

,CBOW,online,skip-gram
0,montr,sa,bred
1,ditl,jeune,cendriyon
2,cendriyon,d'affaires,pulsar
3,genr,mariée,montr
4,dkny,enceinte,pantacourt
5,vet,nue,timex
6,chaussur,adultère,guess
7,pantacourt,assassinée,watx
8,ballerin,fatale,ditl
9,indemod,homme,lorus


### Build Features

#### Our Build Model

In [36]:
data_valid_clean = pd.read_csv("data/cdiscount_valid_clean.csv").fillna("")
data_train_clean = pd.read_csv("data/cdiscount_train_clean.csv").fillna("")

train_array_token_wstem = [line.split(" ") for line in data_train_clean["Description"].values]
valid_array_token_wstem = [line.split(" ") for line in data_valid_clean["Description"].values]

In [37]:
def get_features_mean(lines):
    features = [MODEL[x] for x  in lines if x in MODEL]
    if features == []:   
        fm =np.ones(F_SIZE)
    else :
        fm = np.mean(features,axis=0)
    return fm

def get_matrix_features_means(X):
    X_embedded_ = list(map(get_features_mean, X))
    X_embedded = np.vstack(X_embedded_)
    return X_embedded



In [38]:
for model_name in ["CBOW","skip-gram", "online"]:
    
    if "online" == model_name:
        X_train = train_array_token_wstem
        X_valid = valid_array_token_wstem
    else:
        X_train = train_array_token
        X_valid = valid_array_token
    
    model = model_dic[model_name]
    MODEL = model
    F_SIZE = Features_dimension

    ts = time.time()
    X_embedded_train = get_matrix_features_means(X_train)
    te = time.time()
    t_train = te-ts
    #np.save(embedded_train_dir, X_embedded_train)
    print("Time conversion : %d seconds"%t_train)
    print("Shape Matrix : (%d,%d)"%X_embedded_train.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_train_nb_hash_"+model_name, X_embedded_train)

    
    ts = time.time()
    X_embedded_valid = get_matrix_features_means(X_valid)
    te = time.time()
    t_valid = te-ts
    #np.save(embedded_valid_dir, X_embedded_valid)
    print("Time conversion : %d seconds"%t_valid)
    print("Shape Matrix : (%d,%d)"%X_embedded_valid.shape)
    np.save(DATA_OUTPUT_DIR +"/embedded_valid_nb_hash_"+model_name, X_embedded_valid)
    
    metadata = {"t_train" : t_train, "t_valid" : t_valid, "sg":sg}
    print(metadata)

Time conversion : 19 seconds
Shape Matrix : (95000,300)
Time conversion : 0 seconds
Shape Matrix : (5000,300)
{'t_train': 19.158457040786743, 'sg': 1, 't_valid': 0.9597527980804443}
Time conversion : 17 seconds
Shape Matrix : (95000,300)
Time conversion : 0 seconds
Shape Matrix : (5000,300)
{'t_train': 17.942384958267212, 'sg': 1, 't_valid': 0.9591290950775146}
Time conversion : 15 seconds
Shape Matrix : (95000,300)
Time conversion : 0 seconds
Shape Matrix : (5000,300)
{'t_train': 15.962489128112793, 'sg': 1, 't_valid': 0.8451859951019287}


In [39]:
model["homme"]

array([ -2.68843603e+00,   1.93454778e+00,  -1.81924117e+00,
         2.36969948e+00,   9.32004392e-01,   1.25195098e+00,
         8.65360320e-01,   3.11363190e-01,   6.06419086e-01,
         1.43122864e+00,   1.65650344e+00,  -3.67380810e+00,
         1.87034404e+00,   1.74155509e+00,  -5.33656299e-01,
         1.40374994e+00,   2.30875421e+00,   1.79799902e+00,
        -1.87658596e+00,   1.29379809e+00,   9.08876181e-01,
         1.08965196e-01,  -1.40824032e+00,   8.72196436e-01,
         1.08273089e+00,   7.29817152e-01,  -1.11542833e+00,
         1.52417982e+00,   1.71380103e+00,  -2.30002356e+00,
         1.15143669e+00,  -2.83064008e-01,   2.14521900e-01,
         8.66827071e-01,   7.54454792e-01,   2.86816597e-01,
         5.41400373e-01,  -7.38874137e-01,   5.10816514e-01,
        -1.25636995e+00,  -8.32316935e-01,   3.15392661e+00,
        -6.42400444e-01,  -1.62124023e-01,  -1.09802079e+00,
         1.84242070e+00,  -3.25098783e-01,  -6.75076485e-01,
         1.25538635e+00,

In [40]:
model.wv.syn0[model.wv.vocab["homme"].index] == model["homme"]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [41]:
model.syn1neg.shape

(50130, 300)

In [42]:
vh = model.wv.vocab["homme"]
vh.index

419